In [ ]:
!pip install requests
!pip install bs4
!pip install pandas

import csv
import json

import requests
from bs4 import BeautifulSoup

import pandas as pd

from IPython.display import display, HTML

SITE_ROOT = 'https://www.alltrails.com'
pd.set_option('display.max_colwidth', -1)

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_national_capitals"

r = requests.get(url)

soup = BeautifulSoup(r.content, "html.parser")
table = soup.find_all('table')[1]
rows = table.find_all('tr')
row_list = list()

for tr in rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    row_list.append(row)

row_list

In [6]:
def get_trails(soup):
    trails_ul = soup.find_all("ul", {"data-reactid": "4"})[0]
    for li in trails_ul.find_all("li"):
        for trail_result in li.find_all("div", {"class": "trail-result-card lazyload"}):
            yield trail_result

            
def get_trails_by_attr(trails, attr='itemid'):
    for trail in trails:
        yield trail.attrs[attr]

        
def get_each_trail_link(trails):
    for t in get_trails_by_attr(trails):
        name = t.split('/')[-1]
        link1 = '/'.join([SITE_ROOT, t.lstrip('/')])
        link2 = '/'.join([SITE_ROOT, 'explore', t.lstrip('/')]) + '?ref=sidebar-static-map'
        yield name, link1, link2


def get_trail_data(name, link1, link2):

    r_t = requests.get(link1)
    soup_t = BeautifulSoup(r_t.content, "html.parser")

    detail_data = soup_t.find_all('span', class_="detail-data")
    tag_cloud = soup_t.find('section', class_='tag-cloud').find_all('span', class_='big rounded active')

    data = {
        "thumb": '=IMAGE("{}")'.format(soup_t.find('img', id="static-map-img").attrs['data-src']),
        "name": name,
        "difficulty": soup_t.find('span', class_="diff").get_text(),
        "elevation": soup_t.find('span', class_="detail-data xlate-none").get_text().strip(),
        "distance": detail_data[0].get_text().strip(),
        "description": soup_t.find('p', id="auto-overview").get_text(),
        "link": link2,
        "avg rating": json.loads(soup_t.find('span', {"data-react-class": "TrailRatingStars"}).attrs['data-react-props'])['avgRating'],
        "rewiew count": soup_t.find('span', itemprop="reviewCount").get_text(),
        "route type": detail_data[2].get_text().strip(),
        "tags": ', '.join([tag.get_text() for tag in tag_cloud]),
    }

    return data

In [13]:
url = "https://www.alltrails.com/parks/canada/quebec/mont-orford-national-park"

r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
trails = list(get_trails(soup))
trail_links = list(get_each_trail_link(trails))

display(pd.DataFrame(trail_links))

trails_data_gen = (get_trail_data(*tl) for tl in trail_links)

,0,1,2
0,mont-chauve-trail,https://www.alltrails.com/trail/canada/quebec/mont-chauve-trail,https://www.alltrails.com/explore/trail/canada/quebec/mont-chauve-trail?ref=sidebar-static-map
1,pic-de-lours-sentier-des-cretes-trail,https://www.alltrails.com/trail/canada/quebec/pic-de-lours-sentier-des-cretes-trail,https://www.alltrails.com/explore/trail/canada/quebec/pic-de-lours-sentier-des-cretes-trail?ref=sidebar-static-map
2,mont-chauve-par-le-ruisseau-david,https://www.alltrails.com/trail/canada/quebec/mont-chauve-par-le-ruisseau-david,https://www.alltrails.com/explore/trail/canada/quebec/mont-chauve-par-le-ruisseau-david?ref=sidebar-static-map
3,mont-orford,https://www.alltrails.com/trail/canada/quebec/mont-orford,https://www.alltrails.com/explore/trail/canada/quebec/mont-orford?ref=sidebar-static-map
4,sentier-des-cretes,https://www.alltrails.com/trail/canada/quebec/sentier-des-cretes,https://www.alltrails.com/explore/trail/canada/quebec/sentier-des-cretes?ref=sidebar-static-map
5,mont-alfred-desrocher-et-pic-de-lours-1,https://www.alltrails.com/trail/canada/quebec/mont-alfred-desrocher-et-pic-de-lours-1,https://www.alltrails.com/explore/trail/canada/quebec/mont-alfred-desrocher-et-pic-de-lours-1?ref=sidebar-static-map
6,pekan,https://www.alltrails.com/trail/canada/quebec/pekan,https://www.alltrails.com/explore/trail/canada/quebec/pekan?ref=sidebar-static-map
7,mont-orford-loop,https://www.alltrails.com/trail/canada/quebec/mont-orford-loop,https://www.alltrails.com/explore/trail/canada/quebec/mont-orford-loop?ref=sidebar-static-map
8,des-chenes,https://www.alltrails.com/trail/canada/quebec/des-chenes,https://www.alltrails.com/explore/trail/canada/quebec/des-chenes?ref=sidebar-static-map
9,sentier-du-cerf,https://www.alltrails.com/trail/canada/quebec/sentier-du-cerf,https://www.alltrails.com/explore/trail/canada/quebec/sentier-du-cerf?ref=sidebar-static-map


In [14]:
data = list(trails_data_gen)

In [15]:
df = pd.DataFrame(data)
display(df)

,thumb,name,difficulty,elevation,distance,description,link,avg rating,rewiew count,route type,tags
0,"=IMAGE(""https://cdn-assets.alltrails.com/static-map/production/at-map/13377891/trail-canada-quebec-mont-chauve-trail-at-map-13377891-1539376911-300x250-1.png"")",mont-chauve-trail,MODERATE,368 m,9.2 km,"Mont Chauve Trail is a 9.2 kilometer moderately trafficked loop trail located near Orford, Quebec, Canada that features a lake and is rated as moderate. The trail offers a number of activity options and is best used from March until October. Dogs are also able to use this trail but must be kept on leash.",https://www.alltrails.com/explore/trail/canada/quebec/mont-chauve-trail?ref=sidebar-static-map,4.3,148,Loop,"dogs on leash, kid friendly, hiking, nature trips, trail running, walking, bird watching, forest, lake, river, views, wild flowers"
1,"=IMAGE(""https://cdn-assets.alltrails.com/static-map/production/at-map/24809651/trail-canada-quebec-pic-de-lours-sentier-des-cretes-trail-at-map-24809651-1560375911-300x250-1.png"")",pic-de-lours-sentier-des-cretes-trail,HARD,587 m,11.6 km,"Pic de l'Ours via le sentier des Crêtes is a 11.6 kilometer moderately trafficked point-to-point trail located near Orford, Quebec, Canada that features a great forest setting and is rated as difficult. The trail is primarily used for hiking, walking, snowboarding, and canoeing and is best used from March until October.",https://www.alltrails.com/explore/trail/canada/quebec/pic-de-lours-sentier-des-cretes-trail?ref=sidebar-static-map,4.5,129,Point to Point,"hiking, walking, forest, views, rocky, scramble, no dogs"
2,"=IMAGE(""https://cdn-assets.alltrails.com/static-map/production/at-map/17707172/trail-canada-quebec-mont-chauve-par-le-ruisseau-david-at-map-17707172-1538760296-300x250-1.png"")",mont-chauve-par-le-ruisseau-david,MODERATE,555 m,12.7 km,"Mont Chauve par le Ruisseau-David is a 12.7 kilometer heavily trafficked out and back trail located near Orford, Quebec, Canada that features a lake and is rated as moderate. The trail offers a number of activity options and is best used from April until November.",https://www.alltrails.com/explore/trail/canada/quebec/mont-chauve-par-le-ruisseau-david?ref=sidebar-static-map,4.2,77,Out & Back,"hiking, nature trips, snowshoeing, trail running, bird watching, forest, lake, views, wildlife, muddy, snow, no dogs"
3,"=IMAGE(""https://cdn-assets.alltrails.com/static-map/production/at-map/20166766/trail-canada-quebec-mont-orford-at-map-20166766-1531804690-300x250-1.png"")",mont-orford,HARD,725 m,10.9 km,"Mont Orford is a 10.9 kilometer lightly trafficked out and back trail located near Orford, Quebec, Canada that features a waterfall and is rated as difficult. The trail offers a number of activity options and is best used from May until September.",https://www.alltrails.com/explore/trail/canada/quebec/mont-orford?ref=sidebar-static-map,4.3,67,Out & Back,"hiking, nature trips, rock climbing, trail running, bird watching, forest, river, views, waterfall, wildlife, muddy, rocky, snow, no dogs"
4,"=IMAGE(""https://cdn-assets.alltrails.com/static-map/production/at-map/18241625/trail-canada-quebec-sentier-des-cretes-at-map-18241625-1531802472-300x250-1.png"")",sentier-des-cretes,HARD,919 m,20.8 km,"Sentier des Crêtes is a 20.8 kilometer moderately trafficked loop trail located near Orford, Quebec, Canada that features a lake. The trail is rated as difficult and primarily used for hiking and trail running.",https://www.alltrails.com/explore/trail/canada/quebec/sentier-des-cretes?ref=sidebar-static-map,4.4,30,Loop,"hiking, trail running, lake, views, rocky, no dogs"
5,"=IMAGE(""https://cdn-assets.alltrails.com/static-map/production/at-map/27537989/trail-canada-quebec-mont-alfred-desrocher-et-pic-de-lours-1-at-map-27537989-1565648194-300x250-1.png"")",mont-alfred-desrocher-et-pic-de-lours-1,HARD,572 m,11.3 km,"Mont Alfred Desrocher et Pic de l'Ours 1 is a 11.3 kilometer moderately trafficked out and back trail located nea

In [16]:
# unit example

name, link1, link2 = trail_links[0]
data = get_trail_data(name, link1, link2)
display(data)

{'thumb': '=IMAGE("https://cdn-assets.alltrails.com/static-map/production/at-map/13377891/trail-canada-quebec-mont-chauve-trail-at-map-13377891-1539376911-300x250-1.png")',
 'name': 'mont-chauve-trail',
 'difficulty': 'MODERATE',
 'elevation': '368 m',
 'distance': '9.2 km',
 'description': 'Mont Chauve Trail is a 9.2 kilometer moderately trafficked loop trail located near Orford, Quebec, Canada that features a lake and is rated as moderate. The trail offers a number of activity options and is best used from March until October. Dogs are also able to use this trail but must be kept on leash.',
 'link': 'https://www.alltrails.com/explore/trail/canada/quebec/mont-chauve-trail?ref=sidebar-static-map',
 'avg rating': 4.3,
 'rewiew count': '148',
 'route type': 'Loop',
 'tags': 'dogs on leash, kid friendly, hiking, nature trips, trail running, walking, bird watching, forest, lake, river, views, wild flowers'}